In [1]:
import requests
import json
import pymongo
from bs4 import BeautifulSoup
from pymongo import MongoClient

In [2]:
resp = requests.get('http://moabb.neurotechx.com/docs/datasets.html')

In [3]:
soup = BeautifulSoup(resp.text, 'html.parser')

In [4]:
webpage = soup.find_all('a')

In [5]:
all_datasets_links = webpage[42:74]

In [6]:
all_datasets_links.pop(16) # popping unrelated links

<a class="headerlink" href="#erp-datasets" title="Permalink to this headline">¶</a>

In [7]:
all_datasets_links.pop(23) # popping unrelated links

<a class="headerlink" href="#ssvep-datasets" title="Permalink to this headline">¶</a>

In [8]:
parts_of_links = [tag['href'] for tag in all_datasets_links]

In [9]:
links = ['http://moabb.neurotechx.com/docs/'+link for link in parts_of_links]

In [10]:
moabb_datasets_list = list()

In [11]:
for link in links:
    dataset = dict()
    print('requesting page with URL: '+link)
    dataset_page = requests.get(link)
    dataset_soup = BeautifulSoup(dataset_page.text, 'html.parser')
    dataset['Name of dataset'] = dataset_soup.title.string
    text = dataset_soup.get_text().replace("\n", "")
    if 'Dataset description' in text:
        dataset['Dataset description'] = text[text.index('Dataset description') + len('Dataset description'): text.index('References')]
    else:
        dataset['Dataset description'] = None
    if 'References' in text:
        dataset['References'] = text[text.index('References') + len('References'): text.index('Methods')]
    else:
        dataset['References'] = None
    if 'Methods' in text:
        if 'Examples' in text:
            dataset['Methods'] = text[text.index('Methods') + len('Methods'): text.index('Examples')]
        else:
            dataset['Methods'] = text[text.index('Methods') + len('Methods'): text.index('Back to top')]       
    else:
        dataset['Methods'] = None
    if 'Examples' in text:
        dataset['Examples'] = text[text.index('Examples'): text.index('Back to top')]
    else:
        dataset['Examples'] = None
    dataset['url'] = link
    print('adding data')
    moabb_datasets_list.append(dataset)
    

requesting page with URL: http://moabb.neurotechx.com/docs/generated/moabb.datasets.AlexMI.html#moabb.datasets.AlexMI
adding data
requesting page with URL: http://moabb.neurotechx.com/docs/generated/moabb.datasets.BNCI2014001.html#moabb.datasets.BNCI2014001
adding data
requesting page with URL: http://moabb.neurotechx.com/docs/generated/moabb.datasets.BNCI2014002.html#moabb.datasets.BNCI2014002
adding data
requesting page with URL: http://moabb.neurotechx.com/docs/generated/moabb.datasets.BNCI2014004.html#moabb.datasets.BNCI2014004
adding data
requesting page with URL: http://moabb.neurotechx.com/docs/generated/moabb.datasets.BNCI2015001.html#moabb.datasets.BNCI2015001
adding data
requesting page with URL: http://moabb.neurotechx.com/docs/generated/moabb.datasets.BNCI2015004.html#moabb.datasets.BNCI2015004
adding data
requesting page with URL: http://moabb.neurotechx.com/docs/generated/moabb.datasets.Cho2017.html#moabb.datasets.Cho2017
adding data
requesting page with URL: http://moabb

In [12]:
client = MongoClient('localhost', 27017)

In [13]:
db = client.kerven_moabb_database

In [14]:
moabb_datasets = db.datasets

In [15]:
for dataset in moabb_datasets_list:
    moabb_datasets.insert_one(dataset)

In [16]:
# for dictionary in moabb_datasets_list:
#     with open("moabb_datasets.json", "w") as outfile:
#         json.dump(dictionary, outfile)        

In [17]:
# %notebook kerven_moabb_datasets.ipynb

In [18]:
# %pip install nbformat